In [1]:
import pandas as pd
chlo = "chloride.csv"
sofa = "sofa_pan.csv"
mort = "icu_mort.csv"

sofa = pd.read_csv(sofa)
chl = pd.read_csv(chlo)
mor = pd.read_csv(mort)

In [2]:
mordf = mor[['icustay_id','in_icu_mort']]

In [4]:
chldf = chl[['icustay_id','icu_day','chloride_max']]

# chloride max level in 7 days
chldf = chldf[chldf['icu_day']<=7]
idx = chldf.groupby(['icustay_id'])['chloride_max'].transform(max) == chldf['chloride_max']
df = chldf[idx][['icustay_id','chloride_max']]


In [5]:
sofadf = sofa[['icustay_id','day','sofa']]
sofadf = sofadf[sofadf['day']==1]
sofadf.head()

,icustay_id,day,sofa
0,211552,1.0,7
6,294638,1.0,2
8,228232,1.0,4
12,220597,1.0,5
18,229441,1.0,2


In [6]:
df1 = pd.merge( df, sofadf, on='icustay_id')
df1.head()

,icustay_id,chloride_max,day,sofa
0,211552,114.0,1.0,7
1,294638,108.0,1.0,2
2,228232,102.0,1.0,4
3,220597,107.0,1.0,5
4,229441,106.0,1.0,2


In [7]:
import statsmodels.api as sm

def p_logit(X,y,form='univariate',confounder=None):
    if form=='multivariate':
        logit = sm.Logit(y, pd.concat([X,confounder],axis=1))
        result = logit.fit()
        return result.pvalues[X.columns]
    else:
        pv=pd.Series()
        coef=pd.Series()
        for c in X.columns:
            logit = sm.Logit(y, pd.concat([X[c],confounder],axis=1))
            result = logit.fit()
            pv[c]=result.pvalues[c]
            coef[c]=result.params[c]

    return pv, coef

/Users/panyiheng/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [8]:
df2 = pd.merge(df1, mordf, on='icustay_id')
df2.head()

,icustay_id,chloride_max,day,sofa,in_icu_mort
0,211552,114.0,1.0,7,0
1,294638,108.0,1.0,2,0
2,228232,102.0,1.0,4,0
3,220597,107.0,1.0,5,1
4,229441,106.0,1.0,2,0


In [9]:
X1 = df2[['sofa']]
y1 = df2[['in_icu_mort']]

In [10]:
pv, coef = p_logit(X1, y1)
pv, coef

Optimization terminated successfully.
         Current function value: 0.483055
         Iterations 6


(sofa    0.0
 dtype: float64, sofa   -0.383182
 dtype: float64)

In [11]:
X2 = df2[['chloride_max']]
y2 = df2[['in_icu_mort']]

In [12]:
pv, coef = p_logit(X2, y2)
pv, coef

Optimization terminated successfully.
         Current function value: 0.274293
         Iterations 6


(chloride_max    0.0
 dtype: float64, chloride_max   -0.022936
 dtype: float64)

In [13]:
df2['in_icu_mort'].value_counts()

0    49599
1     4109
Name: in_icu_mort, dtype: int64

In [14]:
import statsmodels.api as sm

X = df2[['sofa', 'chloride_max']]
y = df2[['in_icu_mort']]

logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.243924
         Iterations 7
                         Results: Logit
Model:              Logit            No. Iterations:   7.0000    
Dependent Variable: in_icu_mort      Pseudo R-squared: 0.097     
Date:               2019-02-01 10:15 AIC:              26205.3169
No. Observations:   53708            BIC:              26223.0995
Df Model:           1                Log-Likelihood:   -13101.   
Df Residuals:       53706            LL-Null:          -14509.   
Converged:          1.0000           Scale:            1.0000    
-----------------------------------------------------------------
                 Coef.  Std.Err.     z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------
sofa             0.2669   0.0047   56.5107 0.0000  0.2577  0.2762
chloride_max    -0.0351   0.0003 -115.8874 0.0000 -0.0357 -0.0345



In [41]:
pt = "patient_info.csv"

pt = pd.read_csv(pt)
pt.head()

,Unnamed: 0,subject_id,hadm_id,icustay_id,gender,insurance,ethnicity,admission_type,age
0,0,3,145834,211552,M,Medicare,WHITE,EMERGENCY,77.0
1,1,4,185777,294638,F,Private,WHITE,EMERGENCY,48.0
2,2,6,107064,228232,F,Medicare,WHITE,ELECTIVE,66.0
3,3,9,150750,220597,M,Medicaid,UNKNOWN/NOT SPECIFIED,EMERGENCY,42.0
4,4,11,194540,229441,F,Private,WHITE,EMERGENCY,51.0


In [44]:
dataframe = pd.merge(df2, pt, on='icustay_id')
dataframe = dataframe[['subject_id','hadm_id','icustay_id','chloride_max','sofa','gender','insurance','ethnicity','admission_type','age','in_icu_mort']]
dataframe.head()

,subject_id,hadm_id,icustay_id,chloride_max,sofa,gender,insurance,ethnicity,admission_type,age,in_icu_mort
0,3,145834,211552,114.0,7,M,Medicare,WHITE,EMERGENCY,77.0,0
1,4,185777,294638,108.0,2,F,Private,WHITE,EMERGENCY,48.0,0
2,6,107064,228232,102.0,4,F,Medicare,WHITE,ELECTIVE,66.0,0
3,9,150750,220597,103.0,5,M,Medicaid,UNKNOWN/NOT SPECIFIED,EMERGENCY,42.0,1
4,9,150750,220597,103.0,5,M,Medicaid,UNKNOWN/NOT SPECIFIED,EMERGENCY,42.0,1


In [144]:
def simplify_ethnicity(e):
    if 'WHITE' in e:
        return 'WHITE'
    elif 'ASIAN' in e:
        return 'ASIAN'
    elif 'HISPANIC' in e or 'LATINO' in e:
        return 'HISPANIC'
    elif 'BLACK' in e or 'AFRICAN AMERICAN' in e:
        return 'BLACK'
    else:
        return 'OTHER'

In [146]:
dataframe['ethnicity'] = dataframe['ethnicity'].apply(simplify_ethnicity)
dataframe.head()

,subject_id,hadm_id,icustay_id,chloride_max,sofa,gender,insurance,ethnicity,admission_type,age,in_icu_mort
0,3,145834,211552,114.0,7,M,Medicare,WHITE,EMERGENCY,77.0,0
1,4,185777,294638,108.0,2,F,Private,WHITE,EMERGENCY,48.0,0
2,6,107064,228232,102.0,4,F,Medicare,WHITE,ELECTIVE,66.0,0
3,9,150750,220597,103.0,5,M,Medicaid,OTHER,EMERGENCY,42.0,1
4,9,150750,220597,103.0,5,M,Medicaid,OTHER,EMERGENCY,42.0,1


In [166]:

X1 = dataframe[['sofa', 'chloride_max','gender','insurance','ethnicity','admission_type','age']]
y1 = dataframe[['in_icu_mort']]


In [167]:
dataframe.shape[0]

52941

In [168]:
dataframe.shape[1]

11

In [169]:
dataframe.describe()

,subject_id,hadm_id,icustay_id,chloride_max,sofa,age,in_icu_mort
count,52941.000000,52941.000000,52941.000000,52941.000000,52941.000000,52941.00000,52941.000000
mean,36722.215768,149974.758788,250085.398878,107.422943,3.834438,64.72291,0.075197
std,29007.151334,28914.393946,28883.963793,6.030615,2.973011,17.14040,0.263711
min,3.000000,100001.000000,200001.000000,74.000000,0.000000,19.00000,0.000000
25%,13271.000000,124908.000000,225122.000000,104.000000,1.000000,54.00000,0.000000
50%,26603.000000,150026.000000,250124.000000,107.000000,3.000000,66.00000,0.000000
75%,60020.000000,175057.000000,275080.000000,111.000000,5.000000,78.00000,0.000000
max,99999.000000,199999.000000,299999.000000,198.000000,22.000000,91.40000,1.000000


In [172]:
cat_vars=['gender','insurance','ethnicity','admission_type']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(X1[var], prefix=var)
    X11=X1.join(cat_list)
    X1=X11

X1_vars=X1.columns.values.tolist()
to_keep=[i for i in X1_vars if i not in cat_vars]

In [173]:
X1_final=X1[to_keep]
X1 = X1_final
X1_final.columns.values

array(['sofa', 'chloride_max', 'age', 'gender_F', 'gender_M',
       'insurance_Government', 'insurance_Medicaid', 'insurance_Medicare',
       'insurance_Private', 'insurance_Self Pay', 'ethnicity_ASIAN',
       'ethnicity_BLACK', 'ethnicity_HISPANIC', 'ethnicity_OTHER',
       'ethnicity_WHITE', 'admission_type_ELECTIVE',
       'admission_type_EMERGENCY', 'admission_type_URGENT'], dtype=object)

In [174]:
X1_final_vars=X1_final.columns.values.tolist()
y1= pd.DataFrame(y1,columns=['in_icu_mort'])
X=[i for i in X1_final_vars if i not in y1]
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(X1, y1.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

/Users/panyiheng/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [176]:
import statsmodels.api as sm
logit_model=sm.Logit(y1,X1)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.225238
         Iterations 8
                            Results: Logit
Model:                 Logit             No. Iterations:    8.0000    
Dependent Variable:    in_icu_mort       Pseudo R-squared:  0.156     
Date:                  2019-01-28 12:03  AIC:               23878.6203
No. Observations:      52941             BIC:               24011.7743
Df Model:              14                Log-Likelihood:    -11924.   
Df Residuals:          52926             LL-Null:           -14129.   
Converged:             1.0000            Scale:             1.0000    
----------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z|  [0.025 0.975]
----------------------------------------------------------------------
sofa                      0.2698   0.0050 54.4558 0.0000 0.2601 0.2795
chloride_max              0.0204   0.0027  7.6435 0.0000 0.0152 0.0256
age

/Users/panyiheng/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1029: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
/Users/panyiheng/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/panyiheng/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/panyiheng/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
